# Evaluating `Pointblank`

https://posit-dev.github.io/pointblank/

The following types of tables are supported:

- Polars DataFrame
- Pandas DataFrame
- DuckDB table
- MySQL table
- PostgreSQL table
- SQLite table
- Parquet

It uses `Narwhals` to work with `Polars` and `Pandas` DataFrames and also integrates with `Ibis` to enable the use of `DuckDB`, `MySQL`, `PostgreSQL`, `SQLite`, `Parquet`, and more!

In [1]:
import math
from pathlib import Path

import duckdb
import pandas as pd
import pointblank as pb

In [2]:
TITANIC_URL = 'https://hbiostat.org/data/repo/titanic3.csv'

In [3]:
def get_or_create_db(db_path='titanic.duckdb', url=TITANIC_URL, force_create=False):
    if not Path(db_path).exists() or force_create:
        # Connect to DuckDB and create a persistent database
        con = duckdb.connect(database=db_path, read_only=False)

        # Read the CSV files directly into DuckDB
        con.execute(f"CREATE TABLE IF NOT EXISTS titanic AS SELECT * FROM read_csv('{url}')")
        # Create a new table with a unique identifier
        con.execute("""
            CREATE TABLE titanic_with_id AS 
            SELECT *, ROW_NUMBER() OVER () AS id 
            FROM titanic
        """)
        # Drop the original table and rename the new table
        con.execute("DROP TABLE titanic")
        con.execute("ALTER TABLE titanic_with_id RENAME TO titanic")
    else:
        # Connect to the existing database
        con = duckdb.connect(database=db_path, read_only=False)
    
    return con

In [4]:
con = get_or_create_db()

In [5]:
con.sql("SELECT COUNT(*) FROM titanic")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1309 │
└──────────────┘

In [6]:
con.sql("SELECT * FROM titanic LIMIT 10")

┌────────┬──────────┬─────────────────────────────────────────────────┬─────────┬────────┬───────┬───────┬──────────┬──────────┬─────────┬──────────┬─────────┬───────┬─────────────────────────────────┬───────┐
│ pclass │ survived │                      name                       │   sex   │  age   │ sibsp │ parch │  ticket  │   fare   │  cabin  │ embarked │  boat   │ body  │            home.dest            │  id   │
│ int64  │  int64   │                     varchar                     │ varchar │ double │ int64 │ int64 │ varchar  │  double  │ varchar │ varchar  │ varchar │ int64 │             varchar             │ int64 │
├────────┼──────────┼─────────────────────────────────────────────────┼─────────┼────────┼───────┼───────┼──────────┼──────────┼─────────┼──────────┼─────────┼───────┼─────────────────────────────────┼───────┤
│      1 │        1 │ Allen, Miss. Elisabeth Walton                   │ female  │   29.0 │     0 │     0 │ 24160    │ 211.3375 │ B5      │ S        │ 2       │ 

In [7]:
def create_random_samples(con, n_sample):
    # Get the total number of rows in the titanic table
    total_rows = con.sql("SELECT COUNT(*) FROM titanic").fetchone()[0]
    
    # Calculate the number of rows per sample
    rows_per_sample = math.ceil(total_rows / n_sample)
    
    # Get all rows and shuffle them
    all_rows = con.sql("SELECT * FROM titanic").fetchdf()
    shuffled_rows = all_rows.sample(frac=1).reset_index(drop=True)
    
    samples = []
    for i in range(n_sample):
        offset = i * rows_per_sample
        sample = shuffled_rows.iloc[offset:offset + rows_per_sample]
        samples.append(sample)
    
    return samples

In [8]:
n_sample = 5  # Define the number of samples you want
sample = create_random_samples(con, n_sample)

# Display the first sample to verify
sample[0]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,id
0,3,0,"Bostandyeff, Mr. Guentcho",male,26.0,0,0,349224,7.8958,None,S,None,<NA>,"Bulgaria Chicago, IL",678
1,3,0,"Nancarrow, Mr. William Henry",male,33.0,0,0,A./5. 3338,8.0500,None,S,None,<NA>,None,1052
2,3,1,"Nilsson, Miss. Berta Olivia",female,18.0,0,0,347066,7.7750,None,S,D,<NA>,None,1061
3,2,1,"Laroche, Miss. Louise",female,1.0,1,2,SC/Paris 2123,41.5792,None,C,14,<NA>,Paris / Haiti,479
4,2,1,"Hamalainen, Mrs. William (Anna)",female,24.0,0,2,250649,14.5000,None,S,4,<NA>,"Detroit, MI",429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,0,"Hodges, Mr. Henry Price",male,50.0,0,0,250643,13.0000,None,S,None,149,Southampton,451
258,2,0,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,None,S,None,<NA>,"St Ives, Cornwall / Calumet, MI",346
259,3,0,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,None,S,None,<NA>,None,849
260,3,0,"Rosblom, Mrs. Viktor (Helena Wilhelmina)",female,41.0,0,2,370129,20.2125,None,S,None,<NA>,None,1159


In [9]:
# Fetch the original dataframe from the database
original_df = con.sql("SELECT * FROM titanic").fetchdf()

# Concatenate all samples into a single dataframe
concatenated_samples = pd.concat(sample).sort_values('id').reset_index(drop=True)

# Check if the dataframes are identical
are_identical = original_df.equals(concatenated_samples)

print(f"Are the original dataframe and concatenated samples identical? {'Yes' if are_identical else 'No'}")


Are the original dataframe and concatenated samples identical? Yes


In [10]:
pb.preview(original_df)

GT(_tbl_data=      _row_num_  pclass  survived  \
0             1       1         1   
1             2       1         1   
2             3       1         0   
3             4       1         0   
4             5       1         0   
1304       1305       3         0   
1305       1306       3         0   
1306       1307       3         0   
1307       1308       3         0   
1308       1309       3         0   

                                                 name     sex    age  sibsp  \
0                       Allen, Miss. Elisabeth Walton  female  29.00      0   
1                      Allison, Master. Hudson Trevor    male   0.92      1   
2                        Allison, Miss. Helen Loraine  female   2.00      1   
3                Allison, Mr. Hudson Joshua Creighton    male  30.00      1   
4     Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female  25.00      1   
1304                             Zabour, Miss. Hileni  female  14.50      1   
1305                            Zabour, Miss. Thamine  female    NaN      1   
1306                        Zakarian, Mr. Mapriededer    male  26.50      0   
1307                              Zakarian, Mr. Ortin    male  27.00      0   
1308                               Zimmerman, Mr. Leo    male  29.00      0   

      parch  ticket      fare    cabin embarked  boat  body  \
0         0   24160  211.3375       B5        S     2  <NA>   
1         2  113781  151.5500  C22 C26        S    11  <NA>   
2         2  113781  151.5500  C22 C26        S  None  <NA>   
3         2  113781  151.5500  C22 C26        S  None   135   
4         2  113781  151.5500  C22 C26        S  None  <NA>   
1304      0    2665   14.4542     None        C  None   328   
1305      0    2665   14.4542     None        C  None  <NA>   
1306      0    2656    7.2250     None        C  None   304   
1307      0    2670    7.2250     None        C  None  <NA>   
1308      0  315082    7.8750     None        S  None  <NA>   

                            home.dest    id  
0                        St Louis, MO     1  
1     Montreal, PQ / Chesterville, ON     2  
2     Montreal, PQ / Chesterville, ON     3  
3     Montreal, PQ / Chesterville, ON     4  
4     Montreal, PQ / Chesterville, ON     5  
1304                             None  1305  
1305                             None  1306  
1306                             None  1307  
1307                             None  1308  
1308                             None  1309  , _body=<great_tables._gt_data.Body object at 0x115a83c00>, _boxhead=Boxhead([ColInfo(var='_row_num_', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='right', column_width='41.2px'), ColInfo(var='pclass', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>pclass</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>int64</em></div></div>"), column_align='right', column_width='57px'), ColInfo(var='survived', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>survived</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>int64</em></div></div>"), column_align='right', column_width='72px'), ColInfo(var='name', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>name</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>object</em></div></div>"), column_align='left', column_width='250px'), ColInfo(var='sex', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-s

In [25]:
def validate_titanic_data_sample(sample_df):
    
    validation = (
        pb.Validate(data=sample_df, label="Example Validation")
        .col_vals_not_null("survived")
        .col_vals_not_null("pclass")
        .col_vals_not_null("sex")
        .col_vals_not_null("age")
        .col_vals_not_null("ticket")
        .col_vals_not_null("fare")
        .col_vals_not_null("embarked")
        .col_vals_between("age", 0, 70, na_pass=True)
        .col_vals_between("fare", 0, 500)
        .col_vals_in_set("pclass", {1, 2, 3})
        .col_vals_in_set("embarked", {"C", "Q", "S"})
        .col_vals_in_set("survived", {0, 1})
        .col_vals_in_set("sex", {"male", "female"})
        .interrogate()
    )

    return validation

In [26]:
validate_titanic_data_sample(original_df)

Validate(data=      pclass  survived                                             name  \
0          1         1                    Allen, Miss. Elisabeth Walton   
1          1         1                   Allison, Master. Hudson Trevor   
2          1         0                     Allison, Miss. Helen Loraine   
3          1         0             Allison, Mr. Hudson Joshua Creighton   
4          1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)   
...      ...       ...                                              ...   
1304       3         0                             Zabour, Miss. Hileni   
1305       3         0                            Zabour, Miss. Thamine   
1306       3         0                        Zakarian, Mr. Mapriededer   
1307       3         0                              Zakarian, Mr. Ortin   
1308       3         0                               Zimmerman, Mr. Leo   

         sex    age  sibsp  parch  ticket      fare    cabin embarked  boat  \
0     female  29.00      0      0   24160  211.3375       B5        S     2   
1       male   0.92      1      2  113781  151.5500  C22 C26        S    11   
2     female   2.00      1      2  113781  151.5500  C22 C26        S  None   
3       male  30.00      1      2  113781  151.5500  C22 C26        S  None   
4     female  25.00      1      2  113781  151.5500  C22 C26        S  None   
...      ...    ...    ...    ...     ...       ...      ...      ...   ...   
1304  female  14.50      1      0    2665   14.4542     None        C  None   
1305  female    NaN      1      0    2665   14.4542     None        C  None   
1306    male  26.50      0      0    2656    7.2250     None        C  None   
1307    male  27.00      0      0    2670    7.2250     None        C  None   
1308    male  29.00      0      0  315082    7.8750     None        S  None   

      body                        home.dest    id  
0     <NA>                     St Louis, MO     1  
1     <NA>  Montreal, PQ / Chesterville, ON     2  
2     <NA>  Montreal, PQ / Chesterville, ON     3  
3      135  Montreal, PQ / Chesterville, ON     4  
4     <NA>  Montreal, PQ / Chesterville, ON     5  
...    ...                              ...   ...  
1304   328                             None  1305  
1305  <NA>                             None  1306  
1306   304                             None  1307  
1307  <NA>                             None  1308  
1308  <NA>                             None  1309  

[1309 rows x 15 columns], tbl_name=None, label='Example Validation', thresholds=Thresholds(warning=None, error=None, critical=None), actions=None, lang='en', locale='en')

TODO: Use the first sample to "define" the thresholds and then process the remaining samples with thresholds and actions.

https://posit-dev.github.io/pointblank/user-guide/thresholds.html

https://posit-dev.github.io/pointblank/user-guide/actions.html

In [30]:
for i, s in enumerate(sample):
    validation = validate_titanic_data_sample(s)
    display(f"Sample {i+1} - size: {len(s)} rows")
    display(validation)

'Sample 1 - size: 262 rows'

Validate(data=     pclass  survived                                      name     sex   age  \
0         3         0                 Bostandyeff, Mr. Guentcho    male  26.0   
1         3         0              Nancarrow, Mr. William Henry    male  33.0   
2         3         1               Nilsson, Miss. Berta Olivia  female  18.0   
3         2         1                     Laroche, Miss. Louise  female   1.0   
4         2         1           Hamalainen, Mrs. William (Anna)  female  24.0   
..      ...       ...                                       ...     ...   ...   
257       2         0                   Hodges, Mr. Henry Price    male  50.0   
258       2         0                Berriman, Mr. William John    male  23.0   
259       3         0                   Hansen, Mr. Claus Peter    male  41.0   
260       3         0  Rosblom, Mrs. Viktor (Helena Wilhelmina)  female  41.0   
261       3         0             Boulos, Mrs. Joseph (Sultana)  female   NaN   

     sibsp  parch         ticket     fare cabin embarked  boat  body  \
0        0      0         349224   7.8958  None        S  None  <NA>   
1        0      0     A./5. 3338   8.0500  None        S  None  <NA>   
2        0      0         347066   7.7750  None        S     D  <NA>   
3        1      2  SC/Paris 2123  41.5792  None        C    14  <NA>   
4        0      2         250649  14.5000  None        S     4  <NA>   
..     ...    ...            ...      ...   ...      ...   ...   ...   
257      0      0         250643  13.0000  None        S  None   149   
258      0      0          28425  13.0000  None        S  None  <NA>   
259      2      0         350026  14.1083  None        S  None  <NA>   
260      0      2         370129  20.2125  None        S  None  <NA>   
261      0      2           2678  15.2458  None        C  None  <NA>   

                           home.dest    id  
0               Bulgaria Chicago, IL   678  
1                               None  1052  
2                               None  1061  
3                      Paris / Haiti   479  
4                        Detroit, MI   429  
..                               ...   ...  
257                      Southampton   451  
258  St Ives, Cornwall / Calumet, MI   346  
259                             None   849  
260                             None  1159  
261                   Syria Kent, ON   682  

[262 rows x 15 columns], tbl_name=None, label='Example Validation', thresholds=Thresholds(warning=None, error=None, critical=None), actions=None, lang='en', locale='en')

'Sample 2 - size: 262 rows'

Validate(data=     pclass  survived                                               name  \
262       3         0                    van Billiard, Mr. Austin Blyler   
263       3         0                             Dintcheff, Mr. Valtcho   
264       3         1                     Murphy, Miss. Katherine "Kate"   
265       1         1  Cassebeer, Mrs. Henry Arthur Jr (Eleanor Genev...   
266       3         0                           Risien, Mr. Samuel Beard   
..      ...       ...                                                ...   
519       1         0                          Goldschmidt, Mr. George B   
520       1         1                              Behr, Mr. Karl Howell   
521       1         0                            Artagaveytia, Mr. Ramon   
522       3         0         Strom, Mrs. Wilhelm (Elna Matilda Persson)   
523       3         1                       McCormack, Mr. Thomas Joseph   

        sex   age  sibsp  parch    ticket     fare cabin embarked  boat  body  \
262    male  40.5      0      2  A/5. 851  14.5000  None        S  None   255   
263    male  43.0      0      0    349226   7.8958  None        S  None  <NA>   
264  female   NaN      1      0    367230  15.5000  None        Q    16  <NA>   
265  female   NaN      0      0     17770  27.7208  None        C     5  <NA>   
266    male   NaN      0      0    364498  14.5000  None        S  None  <NA>   
..      ...   ...    ...    ...       ...      ...   ...      ...   ...   ...   
519    male  71.0      0      0  PC 17754  34.6542    A5        C  None  <NA>   
520    male  26.0      0      0    111369  30.0000  C148        C     5  <NA>   
521    male  71.0      0      0  PC 17609  49.5042  None        C  None    22   
522  female  29.0      1      1    347054  10.4625    G6        S  None  <NA>   
523    male   NaN      0      0    367228   7.7500  None        Q  None  <NA>   

               home.dest    id  
262                 None  1265  
263                 None   775  
264                 None  1043  
265         New York, NY    60  
266                 None  1151  
..                   ...   ...  
519         New York, NY   136  
520         New York, NY    23  
521  Montevideo, Uruguay    10  
522                 None  1232  
523                 None  1001  

[262 rows x 15 columns], tbl_name=None, label='Example Validation', thresholds=Thresholds(warning=None, error=None, critical=None), actions=None, lang='en', locale='en')

'Sample 3 - size: 262 rows'

Validate(data=     pclass  survived                                          name     sex  \
524       2         0                            Hickman, Mr. Lewis    male   
525       3         1                      Persson, Mr. Ernst Ulrik    male   
526       1         0                  Nicholson, Mr. Arthur Ernest    male   
527       3         1  Andersson, Mr. August Edvard ("Wennerstrom")    male   
528       3         0                         Skoog, Master. Harald    male   
..      ...       ...                                           ...     ...   
781       3         0                 Bourke, Mrs. John (Catherine)  female   
782       2         0                               Veal, Mr. James    male   
783       1         1                          Marechal, Mr. Pierre    male   
784       3         0    Asplund, Mr. Carl Oscar Vilhelm Gustafsson    male   
785       1         0                         Harrison, Mr. William    male   

      age  sibsp  parch        ticket     fare cabin embarked  boat  body  \
524  32.0      2      0  S.O.C. 14879  73.5000  None        S  None   256   
525  25.0      1      0        347083   7.7750  None        S    15  <NA>   
526  64.0      0      0           693  26.0000  None        S  None   263   
527  27.0      0      0        350043   7.7958  None        S     A  <NA>   
528   4.0      3      2        347088  27.9000  None        S  None  <NA>   
..    ...    ...    ...           ...      ...   ...      ...   ...   ...   
781  32.0      1      1        364849  15.5000  None        Q  None  <NA>   
782  40.0      0      0         28221  13.0000  None        S  None  <NA>   
783   NaN      0      0         11774  29.7000   C47        C     7  <NA>   
784  40.0      1      5        347077  31.3875  None        S  None   142   
785  40.0      0      0        112059   0.0000   B94        S  None   110   

                                home.dest    id  
524  West Hampstead, London / Neepawa, MB   444  
525                                  None  1121  
526                Isle of Wight, England   218  
527                                  None   631  
528                                  None  1207  
..                                    ...   ...  
781                   Ireland Chicago, IL   685  
782              Barre, Co Washington, VT   577  
783                         Paris, France   197  
784                 Sweden  Worcester, MA   645  
785                                  None   151  

[262 rows x 15 columns], tbl_name=None, label='Example Validation', thresholds=Thresholds(warning=None, error=None, critical=None), actions=None, lang='en', locale='en')

'Sample 4 - size: 262 rows'

Validate(data=      pclass  survived                                     name     sex   age  \
786        3         0                  Connaghton, Mr. Michael    male  31.0   
787        2         0          Bryhl, Mr. Kurt Arnold Gottfrid    male  25.0   
788        3         1              Nakid, Miss. Maria ("Mary")  female   1.0   
789        2         0             Parker, Mr. Clifford Richard    male  28.0   
790        3         0                     O'Brien, Mr. Timothy    male   NaN   
...      ...       ...                                      ...     ...   ...   
1043       1         1  Gibson, Mrs. Leonard (Pauline C Boeson)  female  45.0   
1044       3         0                Peacock, Miss. Treasteall  female   3.0   
1045       3         0                     Linehan, Mr. Michael    male   NaN   
1046       1         1              Ryerson, Master. John Borie    male  13.0   
1047       3         0           Laitinen, Miss. Kristina Sofia  female  37.0   

      sibsp  parch              ticket      fare            cabin embarked  \
786       0      0              335097    7.7500             None        Q   
787       1      0              236853   26.0000             None        S   
788       0      2                2653   15.7417             None        C   
789       0      0            SC 14888   10.5000             None        S   
790       0      0              330979    7.8292             None        Q   
...     ...    ...                 ...       ...              ...      ...   
1043      0      1              112378   59.4000             None        C   
1044      1      1  SOTON/O.Q. 3101315   13.7750             None        S   
1045      0      0              330971    7.8792             None        Q   
1046      2      2            PC 17608  262.3750  B57 B59 B63 B66        C   
1047      0      0                4135    9.5875             None        S   

      boat  body                        home.dest    id  
786   None  <NA>             Ireland Brooklyn, NY   725  
787   None  <NA>     Skara, Sweden / Rockford, IL   355  
788      C  <NA>                             None  1049  
789   None  <NA>             St Andrews, Guernsey   528  
790   None  <NA>                             None  1071  
...    ...   ...                              ...   ...  
1043     7  <NA>                     New York, NY   132  
1044  None  <NA>                             None  1113  
1045  None  <NA>                             None   972  
1046     4  <NA>  Haverford, PA / Cooperstown, NY   250  
1047  None  <NA>                             None   944  

[262 rows x 15 columns], tbl_name=None, label='Example Validation', thresholds=Thresholds(warning=None, error=None, critical=None), actions=None, lang='en', locale='en')

'Sample 5 - size: 261 rows'

Validate(data=      pclass  survived                                               name  \
1048       3         1              Nakid, Mrs. Said (Waika "Mary" Mowad)   
1049       3         0                             Barton, Mr. David John   
1050       3         0                                 Foley, Mr. William   
1051       3         1                                     Lang, Mr. Fang   
1052       1         1      Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)   
...      ...       ...                                                ...   
1304       3         0                           Morrow, Mr. Thomas Rowan   
1305       3         0  Lindell, Mrs. Edvard Bengtsson (Elin Gerda Per...   
1306       2         1            West, Mrs. Edwy Arthur (Ada Mary Worth)   
1307       3         0                     Coelho, Mr. Domingos Fernandeo   
1308       2         0  Corey, Mrs. Percy C (Mary Phyllis Elizabeth Mi...   

         sex   age  sibsp  parch              ticket     fare cabin embarked  \
1048  female  19.0      1      1                2653  15.7417  None        C   
1049    male  22.0      0      0              324669   8.0500  None        S   
1050    male   NaN      0      0              365235   7.7500  None        Q   
1051    male  26.0      0      0                1601  56.4958  None        S   
1052  female  56.0      0      1               11767  83.1583   C50        C   
...      ...   ...    ...    ...                 ...      ...   ...      ...   
1304    male   NaN      0      0              372622   7.7500  None        Q   
1305  female  30.0      1      0              349910  15.5500  None        S   
1306  female  33.0      1      2          C.A. 34651  27.7500  None        S   
1307    male  20.0      0      0  SOTON/O.Q. 3101307   7.0500  None        S   
1308  female   NaN      0      0        F.C.C. 13534  21.0000  None        S   

      boat  body                          home.dest    id  
1048     C  <NA>                               None  1051  
1049  None  <NA>               England New York, NY   669  
1050  None  <NA>                            Ireland   805  
1051    14  <NA>                               None   950  
1052     7  <NA>          Mt Airy, Philadelphia, PA   234  
...    ...   ...                                ...   ...  
1304  None  <NA>                               None  1033  
1305     A  <NA>                               None   970  
1306    10  <NA>                Bournmouth, England   594  
1307  None  <NA>                           Portugal   719  
1308  None  <NA>  Upper Burma, India Pittsburgh, PA   383  

[261 rows x 15 columns], tbl_name=None, label='Example Validation', thresholds=Thresholds(warning=None, error=None, critical=None), actions=None, lang='en', locale='en')

In [24]:
for i in [4, 5, 6]:
    display(validation.get_data_extracts(i))

{4:       _row_num_  pclass  survived  \
 15           16       1         0   
 37           38       1         1   
 40           41       1         0   
 46           47       1         0   
 59           60       1         1   
 ...         ...     ...       ...   
 1293       1294       3         0   
 1297       1298       3         0   
 1302       1303       3         0   
 1303       1304       3         0   
 1305       1306       3         0   
 
                                                    name     sex  age  sibsp  \
 15                                  Baumann, Mr. John D    male  NaN      0   
 37        Bradley, Mr. George ("George Arthur Brayton")    male  NaN      0   
 40                            Brewe, Dr. Arthur Jackson    male  NaN      0   
 46                                Cairns, Mr. Alexander    male  NaN      0   
 59    Cassebeer, Mrs. Henry Arthur Jr (Eleanor Genev...  female  NaN      0   
 ...                                                 ...   

{5: Empty DataFrame
 Columns: [_row_num_, pclass, survived, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked, boat, body, home.dest, id]
 Index: []}

{6:       _row_num_  pclass  survived                name   sex   age  sibsp  \
 1225       1226       3         0  Storey, Mr. Thomas  male  60.5      0   
 
       parch ticket  fare cabin embarked  boat  body home.dest    id  
 1225      0   3701   NaN  None        S  None   261      None  1226  }